In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.029934,0.112914,0.335343,0.354057,6052.592985,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.029932,0.112904,0.335286,0.354033,6178.213349,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.029930,0.112893,0.335229,0.354008,6313.892022,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.029928,0.112882,0.335173,0.353984,6459.629005,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029926,0.112872,0.335116,0.353960,6615.424298,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 28 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_bnd1         1028 non-null float64
wp_bnd2         1028 non-null float64
wp_bnd3         1028 non-null float64
wp_bnd7         1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_gpp"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.979697876533
CV Scores:  0.982948413839, 0.983989599044, 0.974964938636, 0.985282451362, 0.981119687758, 0.976858711366, 0.974214590276, 0.975387571986, 0.979646289437, 0.982566511631
OOB score: 0.980052685902
Feature Importances:
('wp_LST.night', 0.33620165140761571)
('air_temp', 0.18247379165836608)
('wp_LST.day', 0.14449485715939997)
('wp_ndvi', 0.076237117922080058)
('PET', 0.072143346518386609)
('VPD', 0.043469508284347279)
('wp_evi', 0.040423756524160311)
('sw_in', 0.038302743232782911)
('wp_lswi2', 0.034876528069452274)
('LW_IN.wp', 0.024210088301700291)
('wind_speed', 0.006596504002568092)
('precip', 0.00057010691914031932)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.977325000004
CV Scores:  0.98503757548, 0.980876247397, 0.977354474809, 0.980226637759, 0.978068499879, 0.975140374495, 0.972007843767, 0.97253326545, 0.976958347509, 0.975046733492
Feature Importances:
('wp_LST.night', 0.14363843975861221)
('wp_evi', 0.13884132364663132)
('wp_lswi2', 0.1206540852117917)
('wp_LST.day', 0.097200524591822685)
('air_temp', 0.096536966008317404)
('wp_ndvi', 0.094184690864718257)
('PET', 0.069760730633800352)
('LW_IN.wp', 0.067129921618636207)
('sw_in', 0.064792556180468031)
('VPD', 0.05463569524611496)
('wind_speed', 0.048672902400481402)
('precip', 0.0039521638386054983)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.820330868686
CV Scores:  0.814064181716, 0.814839500121, 0.78630479109, 0.800931539622, 0.85861992297, 0.871185556522, 0.752974422057, 0.84625574092, 0.837758983384, 0.820374048454


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 2660.62695
Step #501, epoch #50, avg. loss: 130.95221
Step #1001, epoch #100, avg. loss: 19.12914
Step #1501, epoch #150, avg. loss: 10.90339
Step #2001, epoch #200, avg. loss: 7.59950
Step #2501, epoch #250, avg. loss: 5.89042
Step #3001, epoch #300, avg. loss: 4.72776
Step #3501, epoch #350, avg. loss: 4.07379
Step #4001, epoch #400, avg. loss: 3.61226
Step #4501, epoch #450, avg. loss: 3.26641
Step #1, avg. loss: 2181.57666
Step #501, epoch #50, avg. loss: 130.09575
Step #1001, epoch #100, avg. loss: 17.99751
Step #1501, epoch #150, avg. loss: 9.97801
Step #2001, epoch #200, avg. loss: 6.73317
Step #2501, epoch #250, avg. loss: 5.15523
Step #3001, epoch #300, avg. loss: 4.27083
Step #3501, epoch #350, avg. loss: 3.60713
Step #4001, epoch #400, avg. loss: 3.20354
Step #4501, epoch #450, avg. loss: 2.95511
Step #1, avg. loss: 2363.76367
Step #501, epoch #50, avg. loss: 127.59370
Step #1001, epoch #100, avg. loss: 18.19805


TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x119687410>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)